In [11]:
import numpy as np
import pandas as pd
import gym
from gymnasium.wrappers import RecordVideo

from gym import Env, spaces
from stable_baselines3 import PPO
from sklearn.preprocessing import OneHotEncoder

In [12]:
diets = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\diets.csv')
medications = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\medications.csv')
precautions = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\precautions_df.csv')
symptom_severity = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\Symptom-severity.csv')
symptoms = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\symtoms_df.csv')
workouts = pd.read_csv(r'C:\Users\himan\OneDrive\Desktop\Database ML\workout_df.csv')


In [13]:
# Prepare symptoms data
symptoms_long = symptoms.melt(id_vars=['Disease'], value_vars=[f'Symptom_{i}' for i in range(1, 5)], var_name='SymptomType', value_name='Symptom')
symptoms_severity = pd.merge(symptoms_long, symptom_severity, on='Symptom')

In [14]:

# Assuming you have a dataframe symptoms_severity with a column 'Symptom'
encoder = OneHotEncoder(sparse_output=False, drop='first')  # Updated parameter name
encoded_symptoms = encoder.fit_transform(symptoms_severity[['Symptom']])
encoded_symptoms_df = pd.DataFrame(encoded_symptoms, columns=encoder.get_feature_names_out(['Symptom']))
symptoms_severity_encoded = pd.concat([symptoms_severity.drop(['Symptom'], axis=1), encoded_symptoms_df], axis=1)




In [39]:

class HealthcareEnv(Env):
    def __init__(self):
        super(HealthcareEnv, self).__init__()
        self.n_actions = len(diets) + len(medications) + len(workouts)  # Define number of actions
        self.n_features = len(symptoms_severity_encoded.columns)  # Number of features

        # Define action and observation space
        self.action_space = spaces.Discrete(self.n_actions)
        # Ensure the observation space shape is correct: should match the shape of the state
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.n_features,), dtype=np.float32)

        # Initialize state
        self.state = None
        self.done = False

    def reset(self):
    # Sample a random initial state from the encoded data
    self.state = symptoms_severity_encoded.sample(1).iloc[0].values
    print(f"Sampled state: {self.state}")  # Check sampled state
    print(f"State shape: {self.state.shape}")  # Check shape of the sampled state

    # Ensure that the sampled state is numeric
    if isinstance(self.state[0], str):
        raise ValueError("State contains non-numeric values, check the encoding.")

    self.done = False
    return np.array(self.state, dtype=np.float32)


    def step(self, action):
    # Update self.state based on the action (placeholder logic)
    # Here you could modify self.state to represent the effect of the action.
    # For demonstration, let's just simulate a change:
    
    # Assume action impacts the state; replace this logic as needed
    self.state[-1] = max(0, self.state[-1] - 0.1)  # Decrease severity for simplicity

    reward = -self.state[-1]  # Assuming 'Severity' is the last column
    self.done = False  # Adjust this as necessary

    # Return the updated state, reward, and done flag
    return np.array(self.state, dtype=np.float32), reward, self.done, {}




IndentationError: expected an indented block after function definition on line 16 (4242955986.py, line 18)

In [34]:
# Initialize environment
env = HealthcareEnv()


In [35]:
# Define the RL model
model = PPO('MlpPolicy', env, verbose=1)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\himan\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [36]:
print(symptoms_severity_encoded.dtypes)


weight                         int64
Disease_Chicken pox             bool
Disease_Chronic cholestasis     bool
Disease_Drug Reaction           bool
Disease_Fungal infection        bool
Disease_Hepatitis B             bool
Disease_Jaundice                bool
SymptomType_Symptom_1           bool
Symptom_itching                 bool
dtype: object


In [37]:
symptoms_severity_encoded = pd.get_dummies(symptoms_severity)


In [38]:
print(symptoms_severity_encoded.dtypes)

weight                         int64
Disease_Chicken pox             bool
Disease_Chronic cholestasis     bool
Disease_Drug Reaction           bool
Disease_Fungal infection        bool
Disease_Hepatitis B             bool
Disease_Jaundice                bool
SymptomType_Symptom_1           bool
Symptom_itching                 bool
dtype: object


In [31]:
# Train the model
model.learn(total_timesteps=10000)

# Test the model
state = env.reset()
for _ in range(10):
    action, _states = model.predict(state)
    next_state, reward, done, _ = env.step(action)
    print("Action:", action)
    print("Next State:", next_state)
    print("Reward:", reward)
    if done:
        state = env.reset()
    else:
        state = next_state

ValueError: could not broadcast input array from shape (9,) into shape (3,)